#Decorators
Decorators simply take a function as an argument and return a replacement function.
This is can be usefull in a wide range of applications, such as quickly and easily expanding entire functions or even libraries with a single, clear and visible line right at the top of a function.

It is heavily used in frameworks such as [Flask](http://flask.pocoo.org/) (and is in fact the primary way of interacting with the framework). 
So for example (just to see what decorators look like in the real world), we could make a decorator for a website that allows us to easily make sure the user is logged in before allowing them to view the page, otherwise the user will get redirected:

```python
@login_required
@app.route('/food_truck/new/', methods=['GET', 'POST'])
def newFoodTruck():
    """ Create a new food truck """

    credentials = login_session.get('credentials')

    if request.method == 'POST':
        login_user_id = getUserID(login_session['email'])
        newFoodTruck = FoodTruck(
            name=request.form['name'], user_id=login_user_id)
        session.add(newFoodTruck)
        flash('New Food Truck %s Successfully Created' % newFoodTruck.name)
        session.commit()
        return redirect(url_for('showFoodTrucks'))
    else:
        return render_template(
            'newFoodTruck.html', credentials=credentials)
```

So the ```@login_required``` line up there is called a decorator and does the checking and redirecting for us. 

The ```@app.route``` decorate is a built in flask decorator that takes in arguments.

We made ```@login_required``` it like so:

```python
def login_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
         if 'username' not in login_session:
            return redirect(url_for('login', next=request.url))
        return f(*args, **kwargs)
    return decorated_function
```

We will go through how this works (the decorator part, not the Flask part) now:

# Nested Functions
To understand decorators, we first need to understand that functions can be put inside other functions in Python:

In [66]:
def outer():
    print "Hi!"
    def inner():
        print "Hello!"
    return inner()

In [67]:
outer()

Hi!
Hello!


Functions are also just objects, so they can be passed into other functions:

In [68]:
def inner():
    return "Wee!!!"

def outer(func):
    print func()

In [70]:
outer(inner)

Wee!!!


This will be particularly useful later with decorators.

# Function Closures
A function closure simply means that functions defined inside another function (otherwise known as `non-global scope`) rememebers the namespace (or all the variables) of its enclosing function, for when that enclosing function is **defined**.
So for eg:

In [61]:
def outer():
    x = 1
    
    def inner():
        print x
    
    x = 2
    return inner

x = 3

In [62]:
outer()()

2


We can see that x=2 is what the Function Closure remembered.

So here is one academic example of what Function Closures can be used for:

In [10]:
def outer(x):
    def inner():
        print locals()
        print x
    return inner

In [14]:
print1 = outer(1)
print2 = outer(2)

In [15]:
print1()

{'x': 1}
1


In [16]:
print2()

{'x': 2}
2


So we can see that we have created two, unique functions with hard coded values.
This is analogous to having an Constructor (that `outer()` is like), creating different `inner()` objects with different attributes, kind of like so:

In [39]:
class outer():
    def __init__(self, x):
        self.x = x
    def __repr__(self):
        return str(self.x)

In [40]:
print1 = outer(1)
print2 = outer(2)

In [41]:
print1

1

In [42]:
print2

2